<a href="https://colab.research.google.com/github/paulamartingonzalez/GNNs-cox/blob/main/version_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Try with DGL library


In [1]:
!pip install dgl 

In [2]:
!pip install pycox

In [3]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader 

# MNIST is part of torchvision
from torchvision import datasets, transforms
from random import randint
import numpy as np
from dgl.nn import GraphConv
import torchtuples as tt
from pycox.models import LogisticHazard
from pycox.utils import kaplan_meier
from pycox.evaluation import EvalSurv
from torch.utils.data import DataLoader

Using backend: pytorch


## Simulate data

## Time and event (y/n)

In [4]:
def sim_event_times(max_time=700):
    digits = []
    for _ in range(25):
        rnum = randint(0, 10)
        digits.append(float(rnum))
    digits = np.asarray(digits)
    betas = 365 * np.exp(-0.6 * digits) / np.log(1.2)
    event_times = np.random.exponential(betas)
    censored = event_times > max_time
    event_times[censored] = max_time

    return tt.tuplefy(event_times, ~censored)




In [5]:
sim_train = sim_event_times()
sim_test = sim_event_times()

In [6]:
labtrans = LogisticHazard.label_transform(20)
target_train = labtrans.fit_transform(*sim_train)
target_test = labtrans.transform(*sim_test)


### Create random graphs (with 20 nodes and 4 node features)

In [7]:
gs=[]
for i in range(25):
    g = dgl.graph((torch.randint(0, 20, (100,)), torch.randint(0, 20, (100,))), num_nodes=20)
    g = dgl.add_reverse_edges(g)
    g = dgl.remove_self_loop(g)
    #g = dgl.add_self_loop(g)
    g.ndata['h_n'] = torch.randn(g.num_nodes(), 4)
    
    gs.append((g,(torch.tensor(target_train[0][i]),torch.tensor(target_train[1][i])))) ## I don't know if this line is the best, when using tuplefy I was getting errors


In [8]:

gs_test=[]
for i in range(25):
    g = dgl.graph((torch.randint(0, 20, (100,)), torch.randint(0, 20, (100,))), num_nodes=20)
    g = dgl.add_reverse_edges(g)
    g = dgl.remove_self_loop(g)
    #g = dgl.add_self_loop(g)
    g.ndata['h_n'] = torch.randn(g.num_nodes(), 4)
    
    gs_test.append((g,(torch.tensor(target_test[0][i]),torch.tensor(target_test[1][i]))))


gs has shape: (graph,(time,event)),(graph,(time,event)),(graph,(time,event)),...

### Custom collate function

In [10]:
def collate(samples):
    # The input `samples` is a list of pairs
    #  (graph, label).
    graphs, labels = map(list, zip(*samples))
    batched_graph = dgl.batch(graphs)
    return batched_graph, tt.tuplefy(labels[0],labels[1]).stack()

In [11]:
batch = [gs[0], gs[1]]
collate(batch)

(Graph(num_nodes=40, num_edges=374,
       ndata_schemes={'h_n': Scheme(shape=(4,), dtype=torch.float32)}
       edata_schemes={}), (tensor([1, 1]), tensor([1., 1.])))

### Dataloaders

In [12]:

data_loader = DataLoader(gs, batch_size=int(10), shuffle=True,
                            collate_fn=collate)

test_data_loader = DataLoader(gs_test, batch_size=int(10), shuffle=True,
                            collate_fn=collate)

# Training

##  Define model

In [13]:
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)
    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        g.ndata['h'] = h
        return dgl.mean_nodes(g, 'h')

In [18]:
net = GCN(4, 16, 1)
batch = next(iter(data_loader))
x, y = batch
net(x,x.ndata['h_n'].float())

tensor([[-0.2460],
        [-0.1008],
        [-0.0893],
        [ 0.0595],
        [-0.0174],
        [ 0.0321],
        [ 0.0227],
        [ 0.0038],
        [ 0.0576],
        [-0.1647]], grad_fn=<DivBackward0>)

In [20]:
model = LogisticHazard(net, tt.optim.Adam(0.01), duration_index=labtrans.cuts)

In [27]:
batch

(Graph(num_nodes=200, num_edges=1866,
       ndata_schemes={'h_n': Scheme(shape=(4,), dtype=torch.float32), 'h': Scheme(shape=(1,), dtype=torch.float32)}
       edata_schemes={}), (tensor([1, 3]), tensor([1., 1.])))

In [26]:
pred = model.predict(batch[0])
pred.shape

TypeError: ignored

In [29]:
callbacks = [tt.cb.EarlyStopping(patience=5)]
epochs = 50
verbose = True
log = model.fit_dataloader(data_loader, epochs, callbacks, verbose, val_dataloader=test_data_loader)

RuntimeError: ignored